In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json


In [171]:

def get_total_pages(url):
    response = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    
    if response.status_code != 200:
        print(f"Failed to retrieve data from {url}")
        print(response.status_code)
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    page_info = soup.find('span', class_='pageText', attrs={'data-rf-test-name': 'download-and-save-page-number-text'})
    
    if page_info:
        page_text = page_info.text.strip()
        print(f"Page text: {page_text}")
        # Extract the total number of pages using regular expressions
        match = re.search(r'of (\d+)', page_text)
        if match:
            total_pages = int(match.group(1))
            return total_pages
        else:
            print("Total pages not found in text")
            return None
    else:
        print("Page info not found")
        return None

def scrape_redfin(url):
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    soup = BeautifulSoup(response.text, 'html.parser')
    house_containers = soup.find_all('div', class_='HomeCardContainer')

    house_data = []

    for container in house_containers:
        if container.get('aria-label') == 'Advertisement':
            continue

        # Initialize variables for address, geo, and link
        street_address = address_locality = address_region = postal_code = address_country = latitude = longitude = None

        # Extract the address from JSON-LD
        script_tag = container.find('script', type='application/ld+json')
        if script_tag:
            try:
                json_data = json.loads(script_tag.string)
                if isinstance(json_data, list):
                    json_data = json_data[0]  # Use the first item if it's a list
                street_address = json_data['address'].get('streetAddress', '')
                address_locality = json_data['address'].get('addressLocality', '')
                address_region = json_data['address'].get('addressRegion', '')
                postal_code = json_data['address'].get('postalCode', '')
                address_country = json_data['address'].get('addressCountry', '')
                latitude = json_data['geo'].get('latitude', None)
                longitude = json_data['geo'].get('longitude', None)
            except (json.JSONDecodeError, KeyError, TypeError):
                pass

        try:
            # Extract the price
            price = container.find('span', {"class": 'bp-Homecard__Price--value'}).text.strip()
        except AttributeError:
            price = None

        try:
            # Extract the number of beds
            beds = container.find('span', {"class": 'bp-Homecard__Stats--beds'}).text.strip()
        except AttributeError:
            beds = None

        try:
            # Extract the number of baths
            baths = container.find('span', {"class": 'bp-Homecard__Stats--baths'}).text.strip()
        except AttributeError:
            baths = None

        try:
            # Extract the square footage
            sqft = container.find('span', {"class": 'bp-Homecard__LockedStat--value'}).text.strip()
        except AttributeError:
            sqft = None

        try:
            # Extract the link
            link = container.find('a', {"class": 'link-and-anchor'})['href']
            full_link = f"https://www.redfin.com{link}"
        except (AttributeError, TypeError):
            full_link = None

        # Append the extracted data to the list
        house_data.append({
            'price': price,
            'beds': beds,
            'baths': baths,
            'sqft': sqft,
            'address': f"{street_address}, {address_locality}, {address_region} {postal_code}, {address_country}",
            'street': street_address,
            'locality': address_locality,
            'region': address_region,
            'zip': postal_code,
            'country': address_country,
            'latitude': latitude,
            'longitude': longitude,
            'link': full_link
        })

    return house_data


# Function to construct the URL based on the selected filters
def redfin_url_filter(filters):
    filter_params = []
    
    # Property type
    property_types = []
    for prop_type, enabled in filters["property_type"].items():
        if enabled:
            property_types.append(prop_type)
    if property_types:
        filter_params.append(f"property-type={'+'.join(property_types)}")
    
    # Price filters
    if "min_price" in filters and filters["min_price"] is not None:
        filter_params.append(f"min-price={filters['min_price']}")
    if "max_price" in filters and filters["max_price"] is not None:
        filter_params.append(f"max-price={filters['max_price']}")
    
    # Beds filters
    if "min_beds" in filters and filters["min_beds"] is not None:
        filter_params.append(f"min-beds={filters['min_beds']}")
    if "max_beds" in filters and filters["max_beds"] is not None:
        filter_params.append(f"max-beds={filters['max_beds']}")
    
    # Baths filters
    if "min_baths" in filters and filters["min_baths"] is not None:
        filter_params.append(f"min-baths={filters['min_baths']}")
    if "max_baths" in filters and filters["max_baths"] is not None:
        filter_params.append(f"max-baths={filters['max_baths']}")
    
    # Construct the final URL
    filter_string = ",".join(filter_params)
    return f"/filter/{filter_string}"

## Begin Script using functions above

In [195]:
# City = "Peoria"
# State = "IL"
zip = 98177

base_url = "https://www.redfin.com/zipcode/"

filters = {
  "property_type": {
        "house": True,
        "townhouse": True,
        "condo": False,
        "multifamily": False,
        "land": False,
        "manufactured": False,
        "co-op": False
    },
    "min_price": None,
    "max_price": None,
    "min_beds": None,
    "max_beds": None,
    "min_baths": None,
    "max_baths": None
}


# Example usage
url_filters = redfin_url_filter(filters)


print(base_url)
print(f"{base_url}{url_filters}")

zips = pd.read_csv("zip_code_database.csv")

if zip == None:

  target_zips = zips[(zips["primary_city"] == City) & (zips["state"] == State)]["zip"].tolist()
else:
    target_zips = [zip]


print(target_zips)

data = []


for zip in target_zips:
    url = f"{base_url}{zip}{url_filters}"
    
    try:
        total_pages = get_total_pages(url)
        if total_pages is None:
            print(f"No pages found for ZIP code {zip}. Skipping.")
            continue
    except Exception as e:
        print(f"Error getting total pages for ZIP code {zip}: {e}")
        continue
    for page_number in range(1,total_pages+1):
        print(zip, " Page: ", page_number)
        if page_number == 1:
            page_url = url
        else:
            page_url = f"{url}/page-{page_number}"

        print(page_url)

        scraped_data = scrape_redfin(page_url)
        
        for record in scraped_data:
            record['search_zip'] = str(zip)

        data.extend(scraped_data)

df = pd.DataFrame(data)

df = df[df['zip'] == df['search_zip']]

df.head()

https://www.redfin.com/zipcode/
https://www.redfin.com/zipcode//filter/property-type=house+townhouse
[98177]
Page text: Viewing page 1 of 1
98177  Page:  1
https://www.redfin.com/zipcode/98177/filter/property-type=house+townhouse


,price,beds,baths,sqft,address,street,locality,region,zip,country,latitude,longitude,link,search_zip
0,"$3,360,000",5 beds,5.5 baths,"3,456","861 NW 175th St, Shoreline, WA 98177, US",861 NW 175th St,Shoreline,WA,98177,US,47.755255,-122.368715,https://www.redfin.com/WA/Shoreline/861-NW-175...,98177
1,"$975,000",3 beds,2.5 baths,"1,750","20316 3rd Ave NW, Shoreline, WA 98177, US",20316 3rd Ave NW,Shoreline,WA,98177,US,47.777130,-122.361046,https://www.redfin.com/WA/Shoreline/20316-3rd-...,98177
2,"$925,000",3 beds,2 baths,"1,720","208 NW 177th St, Shoreline, WA 98177, US",208 NW 177th St,Shoreline,WA,98177,US,47.757613,-122.360175,https://www.redfin.com/WA/Shoreline/208-NW-177...,98177
3,"$1,600,000",2 beds,2 baths,"2,235","16732 16th Ave NW, Shoreline, WA 98177, US",16732 16th Ave NW,Shoreline,WA,98177,US,47.751582,-122.378856,https://www.redfin.com/WA/Shoreline/16732-16th...,98177
4,"$1,325,000",4 beds,3.5 baths,"3,070","138 NW 143rd St, Seattle, WA 98177, US",138 NW 143rd St,Seattle,WA,98177,US,47.732626,-122.359516,https://www.redfin.com/WA/Seattle/138-NW-143rd...,98177


# Try to use stingray to get the same data


In [188]:
#House Initial Info API

house = df['link'].str.replace(r'^https?://[^/]+', '', regex=True)[1]

print(house)

stingray_url = f"http://www.redfin.com/stingray/api/home/details/initialInfo?path={house}"

print(stingray_url)

/WA/Shoreline/20316-3rd-Ave-NW-98177/home/78377
http://www.redfin.com/stingray/api/home/details/initialInfo?path=/WA/Shoreline/20316-3rd-Ave-NW-98177/home/78377


In [191]:
#Full House API (needs listing and property IDS. Can be gotten from the Initial Info API)

pid = 78377

lid = 188760919

new_url = f"https://www.redfin.com/stingray/api/home/details/belowTheFold?propertyId={pid}&accessLevel=1&listingId={lid}"

print(new_url)


https://www.redfin.com/stingray/api/home/details/belowTheFold?propertyId=78377&accessLevel=1&listingId=188760919



https://github.com/alientechsw/RedfinPlus/blob/master/docs/REDFIN.md

In [3]:
# GIS Search API

import requests

zip = 98117

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
# Base URL
search_api = "https://www.redfin.com/stingray/api/gis"
query_location_api = f"https://www.redfin.com/stingray/do/query-location?location={zip}&v=2"

response = requests.get(query_location_api, headers=headers) 
soup = BeautifulSoup(response.text, 'html.parser').text
prefix_removed = soup.split('&&', 1)[1]
data = json.loads(prefix_removed)
region_id = data["payload"]["exactMatch"].get("id").split("_",1)[1]


# Default parameters
params = {
    #??Active Listings
    "al": 1,
    #Include Nearby Homes
    "include_nearby_homes": False,
    # Market. ie Seattle
    "market": None,
    # Number of homes to retrieve
    "num_homes": 200,
    #How to Sort the homes
    "ord": "days-on-redfin-asc",
    "page_number": 1,
    "poly": None,
    #Listing Types
    "sf": "1,2,3,4,5,6,7",
    "start": None,
    "status": 9,
    # User input property types
    "uipt": "1,2,3,4,5,6,7,8",
    # ??API Version?
    "v": 8,
    "zoomLevel": None,
    #Type of Region analyzed
    "region_type" : 2,
    "region_id" : region_id
    
}


# Function to convert parameters dictionary to query string format
def dict_to_query_string(params):
    return "&".join(f"{key}={value}" for key, value in params.items() if params.get(key) != None)

stingray_search_url = f"{search_api}?{dict_to_query_string(params)}"

print(stingray_search_url)


https://www.redfin.com/stingray/api/gis?al=1&include_nearby_homes=False&num_homes=200&ord=days-on-redfin-asc&page_number=1&sf=1,2,3,4,5,6,7&status=9&uipt=1,2,3,4,5,6,7,8&v=8&region_type=2&region_id=40830


In [130]:
def parse_json(data):
    homes = data.get('payload', {}).get('homes', [])
    print(homes)
    parsed_homes = []
    
    for home in homes:
        home_info = {
            "MLS ID": home.get('mlsId', {}).get('value'),
            "Status": home.get('mlsStatus'),
            "Price": home.get('price', {}).get('value'),
            "HOA Fee": home.get('hoa', {}).get('value'),
            "Square Feet": home.get('sqFt', {}).get('value'),
            "Price per Square Foot": home.get('pricePerSqFt', {}).get('value'),
            "Lot Size": home.get('lotSize', {}).get('value'),
            "Bedrooms": home.get('beds'),
            "Bathrooms": home.get('baths'),
            "Location": home.get('location', {}).get('value'),
            "Stories": home.get('stories'),
            "Address": home.get('streetLine', {}).get('value'),
            "City": home.get('city'),
            "State": home.get('state'),
            "ZIP Code": home.get('postalCode', {}).get('value'),
            "Year Built": home.get('yearBuilt', {}).get('value'),
            "URL": home.get('url'),
        }
        parsed_homes.append(home_info)
    
    return parsed_homes



response = requests.get(stingray_search_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
soup = BeautifulSoup(response.text, 'html.parser').text
prefix_removed = soup.split('&&', 1)[1]

data = json.loads(prefix_removed)

print(data)


homes = parse_json(data)

print(homes)

{'version': 538, 'errorMessage': 'Success', 'resultCode': 0, 'payload': {'homes': [{'mlsId': {'label': 'MLS#', 'value': '2245837'}, 'showMlsId': False, 'mlsStatus': 'Active', 'showDatasourceLogo': True, 'price': {'value': 870000, 'level': 1}, 'hideSalePrice': False, 'hoa': {'level': 1}, 'isHoaFrequencyKnown': True, 'sqFt': {'value': 1760, 'level': 1}, 'pricePerSqFt': {'value': 494, 'level': 1}, 'lotSize': {'value': 4141, 'level': 1}, 'beds': 2, 'baths': 1.0, 'fullBaths': 1, 'location': {'value': 'Loyal Heights', 'level': 1}, 'stories': 1.0, 'latLong': {'value': {'latitude': 47.6865946, 'longitude': -122.3824591}, 'level': 1}, 'streetLine': {'value': '7755 NW 20th Ave', 'level': 1}, 'unitNumber': {'level': 1}, 'city': 'Seattle', 'state': 'WA', 'zip': '98117', 'postalCode': {'value': '98117', 'level': 1}, 'countryCode': 'US', 'showAddressOnMap': True, 'soldDate': 1185865200000, 'searchStatus': 1, 'propertyType': 6, 'uiPropertyType': 1, 'listingType': 1, 'propertyId': 164389, 'listingId':

In [131]:

df = pd.DataFrame(homes)

df.dtypes
# display(df)

df.groupby("ZIP Code").count()

,MLS ID,Status,Price,HOA Fee,Square Feet,Price per Square Foot,Lot Size,Bedrooms,Bathrooms,Location,Stories,Address,City,State,Year Built,URL
ZIP Code,,,,,,,,,,,,,,,,
98117,79,79,79,29,76,76,70,76,76,79,76,79,79,79,72,79
98177,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1


In [133]:
display(df)

,MLS ID,Status,Price,HOA Fee,Square Feet,Price per Square Foot,Lot Size,Bedrooms,Bathrooms,Location,Stories,Address,City,State,ZIP Code,Year Built,URL
0,2245837,Active,870000,NaN,1760.0,494.0,4141.0,2.0,1.0,Loyal Heights,1.0,7755 NW 20th Ave,Seattle,WA,98117,1942.0,/WA/Seattle/7755-20th-Ave-NW-98117/home/164389
1,2245612,Active,1399000,NaN,2560.0,546.0,4080.0,5.0,4.0,Loyal Heights,2.0,7337 27th Ave NW,Seattle,WA,98117,1928.0,/WA/Seattle/7337-27th-Ave-NW-98117/home/164965
2,2244189,Active,1175000,NaN,2140.0,549.0,4080.0,3.0,3.0,Loyal Heights,1.5,7024 17th Ave NW,Seattle,WA,98117,1929.0,/WA/Seattle/7024-17th-Ave-NW-98117/home/164588
3,2245686,Active,1099999,NaN,1923.0,572.0,1676.0,4.0,2.5,Crown Hill,1.5,8747 14th Ave NW Unit D,Seattle,WA,98117,2021.0,/WA/Seattle/8747-14th-Ave-NW-98117/unit-D/home...
4,2241623,Active,899950,17.0,1869.0,482.0,4234.0,2.0,2.0,Ballard,2.0,6755 25th Ave NW,Seattle,WA,98117,1903.0,/WA/Seattle/6755-25th-Ave-NW-98117/home/164467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2205867,Active,995000,NaN,1490.0,668.0,5400.0,3.0,2.0,Crown Hill,1.5,9230 13th Ave NW,Seattle,WA,98117,1947.0,/WA/Seattle/9230-13th-Ave-NW-98117/home/96966
76,1AFE55ED4812,Active,649950,NaN,1171.0,555.0,NaN,2.0,1.0,Seattle,4.0,9013C Plan,Seattle,WA,98117,NaN,/WA/Seattle/Seattle/9013C/home/189914759
77,2191647,Active,924900,24.0,1850.0,500.0,1730.0,3.0,2.0,Whittier,1.5,8035 A Mary Ave NW,Seattle,WA,98117,2024.0,/WA/Seattle/8035-Mary-Ave-NW-98117/unit-A/home...
78,2187444,Active,170000,NaN,NaN,NaN,6900.0,NaN,NaN,Greenwood,NaN,0 xxxx 3rd Ave NW,Seattle,WA,98117,NaN,/WA/Seattle/0-xxxx-3rd-Ave-NW-98117/home/18902...
